In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
import cPickle as pickle

In [2]:
with open('./kinopoisk_train.csv') as f_in:
    y = []
    texts = []
    for line in f_in:
        movie_id, sentiment, text = line.strip('\n').split('\t', 3)
        if sentiment=='negative' or sentiment=='positive':
            y.append(1 if sentiment=='positive' else 0)
            texts.append(text)

In [3]:
%%time
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(texts)
print X.shape

(130181, 549081)
CPU times: user 1min 20s, sys: 2.8 s, total: 1min 23s
Wall time: 1min 24s


In [4]:
%%time
y_pred_cv = cross_val_predict(LogisticRegression(), X, y, cv=4, n_jobs=-1, verbose=True, method='predict_proba')
print 'Cross-validated AUC: %.4f' % roc_auc_score(y, y_pred_cv[:,1])

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   59.1s finished


Cross-validated AUC: 0.9707
CPU times: user 3.79 s, sys: 294 ms, total: 4.08 s
Wall time: 59.4 s


In [5]:
%%time
L = LogisticRegression()
L.fit(X, y)

features = tfidf.get_feature_names()
features_with_weights = pd.DataFrame(zip(features, L.coef_[0]), columns=['feature', 'weight']).sort_values('weight')

CPU times: user 22.7 s, sys: 343 ms, total: 23 s
Wall time: 23.4 s


In [6]:
with open('logreg.pkl', 'wb') as f_out:
    pickle.dump(L, f_out)
with open('tf-idf.pkl', 'wb') as f_out:
    pickle.dump(tfidf, f_out)

In [15]:
#L.predict('tttt')

ValueError: X has 1 features per sample; expecting 549081